In [ ]:
import os
import streamlit as st
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chat_models import AzureChatOpenAI as AzureOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.agents import Tool, initialize_agent
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
load_dotenv()
 # Define OpenAI credentials
openai_api_type = os.getenv("OPENAI_API_TYPE")
openai_api_version = os.getenv("OPENAI_API_VERSION")
openai_api_base = os.getenv("OPENAI_API_BASE")
openai_api_key = os.getenv("OPENAI_API_KEY")
model_deployment_id = os.getenv("MODEL_DEPLOYMENT_ID")
# Set environment variables
if openai_api_base is not None:
    os.environ["OPENAI_API_BASE"] = openai_api_base
if openai_api_key is not None:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if openai_api_type is not None:
    os.environ["OPENAI_API_TYPE"] = openai_api_type
if openai_api_version is not None:
     os.environ["OPENAI_API_VERSION"] = str(openai_api_version)

In [ ]:
embeddings = OpenAIEmbeddings(
client=any,
deployment='embeddings',
model='text-embedding-ada-002',
openai_api_type="azure",
openai_api_base="https://copilot-openai.openai.azure.com",
openai_api_version="2023-05-15",
openai_api_key=os.environ["OPENAI_API_KEY"],
chunk_size=1
)

In [ ]:
uploaded_docs = FAISS.load_local("./faiss_uploaded_docs", embeddings)
llm1 = AzureOpenAI(deployment_name='trouble-buddy', model_name='gpt-3.5-turbo', temperature=0, client=any)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

qa1 = RetrievalQA.from_chain_type(
    llm=llm1,
    retriever=uploaded_docs.as_retriever(),
    memory=memory
)

In [ ]:
kb_sre = FAISS.load_local("./site_reliability_engineering", embeddings)
#llm = AzureOpenAI(model_kwargs={'engine':'trouble-buddy'}, client=any)
llm = AzureOpenAI(deployment_name='trouble-buddy', model_name='gpt-3.5-turbo', temperature=0.0, client=any)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=kb_sre.as_retriever(),
    chain_type='stuff',
    memory=memory
)


In [ ]:
uploaded_docs.similarity_search("What happened on Elastic Cloud?")

In [ ]:
qa1.run("What happened?")

In [ ]:
qa1.run("for how long did the incident last?")

In [ ]:
tools = [
    Tool(
        name="Documentos enviados",
        func=qa1.run,
        description=(
            'Use this tool to answer general purpose questions'
        )
    ),  
    Tool(
        name="Base de Conhecimento sobre o livro Building Secure and Reliable Systems do Google",
        func=qa.run,
        description=(
            'Only Use this tool when the user asks to include Site Reliability Engineering (SRE) topics as part of the answer'
        )
    )
]

In [ ]:
system_template = """
Complete the objective as best you can. You have access to the following tools:
- "Documentos enviados"
- "Base de Conhecimento sobre o livro Building Secure and Reliable Systems do Google"
- "Base de Conhecimento sobre o livro Site Reliability Engineering do Google"
The user question is: {user_input}

You are helpful and cheerful assistant who is capable of providing insightful and precise answers to user's question based on the information available in the faiss_uploaded_docs index.
You try to be as helpeful as possible but if you do not know the answer, do not invent an it instead say "I do not know how to answer this question based on the context provided".
As a complement, if asked to corralate the answer with SRE practices or resilient system's design, try to expand on the answer by providing insights extracted from the the site_reliability_engineering and sre_building_secure_and_reliable_systems indexes on how SRE practices and the design of resilient systems could help in the future.
When providing insights to the user, always try to provide at least 3 insights and cite the sources.
These were previous tasks you completed:
Your answer is:
"""

In [ ]:
system_template = """
Complete the objective as best you can. You have access to the following tools:
- "Documentos enviados"
- "Base de Conhecimento sobre o livro Building Secure and Reliable Systems do Google"
- "Base de Conhecimento sobre o livro Site Reliability Engineering do Google"
The user question is: {user_input}

You are helpful and cheerful assistant who is capable of providing insightful and precise answers to user's question based on the information available in the faiss_uploaded_docs index.
You try to be as helpeful as possible but if you do not know the answer, do not invent an it instead say "I do not know how to answer this question based on the context provided".
As a complement, if asked to corralate the answer with SRE practices or resilient system's design, try to expand on the answer by providing insights extracted from the the site_reliability_engineering and sre_building_secure_and_reliable_systems indexes on how SRE practices and the design of resilient systems could help in the future.
When providing insights to the user, always try to provide at least 3 insights and cite the sources.
These were previous tasks you completed:
Your answer is:
"""

In [ ]:
human_template = f"""
according to my question {user_input}, can you provide a detailed explanation of the event that happened? with additional insights on how SRE practices and the design of resilient systems could help in the future?
"""

In [ ]:
prompt = PromptTemplate(template=system_template, input_variables=['user_input'],)

In [ ]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [32]:
messages = [
    SystemMessage(content=system_template),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="What happened on Elastic Cloud?"),
]

In [33]:
res = llm(messages)

In [34]:
res.content

'I do not know how to answer this question based on the context provided. Could you please provide more information or context about what happened on Elastic Cloud?'

In [ ]:
agente = initialize_agent(
        agent='chat-zero-shot-react-description',
        tools=tools,
        llm=llm,
        verbose=True,
        max_interactions=3,
        early_stopping_method='generate',
        handle_parsing_errors=True,
        memory=memory,
    )


d = agente.run(prompt.format_prompt(user_input="What happened on Elastic Cloud"))
#print(d[0]['messsage'])
#return d[0]["message"]

In [ ]:
d = prompt.format_prompt(user_input="What happened on Elastic Cloud")

agente.run(d)
    
